<a href="https://colab.research.google.com/github/minzero-1003/capstone-project/blob/main/11_09_10k_data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gemma 사용을 위해 Hugging Face 트랜스포머 라이브러리 및 관련 의존성 설치
!pip install transformers accelerate bitsandbytes
# ClinicalTrials.gov API 호출 및 데이터 처리를 위한 필수 라이브러리
!pip install requests pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 16.6 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd

print("⏳ ClinicalTrials.gov API를 호출하여 데이터 10,000개를 가져옵니다...")

base_url = "https://clinicaltrials.gov/api/v2/studies"
params = {
    "sort": "LastUpdatePostDate:desc",
    "pageSize": 10000
}

try:
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    data = response.json()

    # eligibilityCriteria 텍스트만 추출
    criteria_list = []
    for study in data.get('studies', []):
        try:
            criteria = study['protocolSection']['eligibilityModule']['eligibilityCriteria']
            criteria_list.append({'eligibilityCriteria': criteria})
        except KeyError:
            continue

    print(f"✅ API로부터 {len(criteria_list)}개의 'eligibilityCriteria' 텍스트를 성공적으로 가져왔습니다.")

    # 결과 저장: trial_embeddings_10k.csv의 재료
    trials_df = pd.DataFrame(criteria_list)
    trials_df.to_csv("trials_api_10k.csv", index=False, encoding='utf-8-sig')
    print("✅ 'trials_api_10k.csv' 파일 저장 완료!")

except Exception as e:
    print(f"🔥🔥 API 호출 또는 데이터 처리 중 오류 발생: {e}")

⏳ ClinicalTrials.gov API를 호출하여 데이터 10,000개를 가져옵니다...
✅ API로부터 997개의 'eligibilityCriteria' 텍스트를 성공적으로 가져왔습니다.
✅ 'trials_api_10k.csv' 파일 저장 완료!


In [ ]:
!pip install sentence-transformers tqdm

from sentence_transformers import SentenceTransformer
import pandas as pd

# 사용할 임베딩 모델 로드 (프롬프트에서 지정된 모델)
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# --- 1. Trial 데이터 임베딩 (10,000개) ---
print("\n⏳ 'trials_api_10k.csv' 파일을 임베딩합니다...")
df_trial = pd.read_csv("trials_api_10k.csv")
texts_trial = df_trial['eligibilityCriteria'].dropna().tolist()
embeddings_trial = model.encode(texts_trial, show_progress_bar=True)

# 최종 파일 생성: trial_embeddings_10k.csv
embedding_df_trial = pd.DataFrame(embeddings_trial)
text_df_trial = pd.DataFrame(texts_trial, columns=['original_text'])
final_df_trial = pd.concat([text_df_trial, embedding_df_trial], axis=1)
final_df_trial.to_csv("trial_embeddings_10k.csv", index=False)
print("✅ 'trial_embeddings_10k.csv' 파일 생성 완료!")

# --- 2. Orphanet 데이터 임베딩 (전체) ---
print("\n⏳ 'out4.csv' 파일을 임베딩합니다...")
df_hpo = pd.read_csv("/content/out4.csv")
texts_hpo = df_hpo['HPOTERM'].dropna().tolist()
embeddings_hpo = model.encode(texts_hpo, show_progress_bar=True)

# 최종 파일 생성: orphanet_embeddings_full.csv
embedding_df_hpo = pd.DataFrame(embeddings_hpo)
text_df_hpo = pd.DataFrame(texts_hpo, columns=['original_hpoterm'])
final_df_hpo = pd.concat([text_df_hpo, embedding_df_hpo], axis=1)
final_df_hpo.to_csv("orphanet_embeddings_full.csv", index=False)
print("✅ 'orphanet_embeddings_full.csv' 파일 생성 완료!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...


⏳ 'trials_api_10k.csv' 파일을 임베딩합니다...


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

✅ 'trial_embeddings_10k.csv' 파일 생성 완료!

⏳ 'out4.csv' 파일을 임베딩합니다...


Batches:   0%|          | 0/3614 [00:00<?, ?it/s]

✅ 'orphanet_embeddings_full.csv' 파일 생성 완료!


In [ ]:
import pandas as pd
import random
import re
from tqdm import tqdm

print("⏳ 1단계: 원본 CSV 파일들을 불러옵니다...")
trials_df = pd.read_csv("/content/trials_api_10k.csv")
hpo_df = pd.read_csv("/content/out4.csv")
print("✅ 파일 로드 완료!")

trials_criteria = trials_df['eligibilityCriteria'].dropna().tolist()
hpo_terms = hpo_df['HPOTERM'].dropna().tolist()

training_data = []
seen_pairs = set()

# --- 긍정 예시 (Label = 1) 생성 ---
print("  - 긍정 데이터 생성 중 (포함 관계 확인)...")
# 정규 표현식을 사용해 criteria에 hpo_term이 포함되는 긍정 쌍 빠르게 찾기
term_pattern = '|'.join(re.escape(term) for term in hpo_terms if len(term) >= 3)
regex = re.compile(term_pattern, re.IGNORECASE)

for criteria in tqdm(trials_criteria, desc="긍정 데이터 검색 중"):
    found_terms = regex.findall(criteria)
    for term in found_terms:
        pair = (criteria, term)
        if pair not in seen_pairs:
            training_data.append({'text_A': criteria, 'text_B': term, 'label': 1})
            seen_pairs.add(pair)

num_positive_samples = len([d for d in training_data if d['label']==1])
print(f"  - ✅ 긍정 데이터 {num_positive_samples}개 생성 완료!")

# --- 부정 예시 (Label = 0) 생성 (1:1 비율) ---
print(f"  - 부정 데이터 생성 중 (목표: {num_positive_samples}개)")
NUM_NEGATIVE_SAMPLES = num_positive_samples # 1:1 비율 설정

negative_count = 0
with tqdm(total=NUM_NEGATIVE_SAMPLES, desc="부정 데이터 생성 중") as pbar:
    while negative_count < NUM_NEGATIVE_SAMPLES:
        random_criteria = random.choice(trials_criteria)
        random_term = random.choice(hpo_terms)

        # 부정 예시 조건: 3글자 이상이고, 포함되지 않아야 함
        if len(random_term) >= 3 and random_term.lower() in random_criteria.lower():
            continue

        pair = (random_criteria, random_term)
        if pair not in seen_pairs:
            training_data.append({'text_A': random_criteria, 'text_B': random_term, 'label': 0})
            seen_pairs.add(pair)
            negative_count += 1
            pbar.update(1)
print(f"  - ✅ 부정 데이터 {negative_count}개 생성 완료!")

# --- 최종 CSV 파일로 저장 ---
print("\n⏳ 3단계: 'training_data_10k.csv' 파일로 저장합니다...")
final_df = pd.DataFrame(training_data)
final_df = final_df.sample(frac=1).reset_index(drop=True) # 데이터 섞기
final_df.to_csv("training_data_10k.csv", index=False, encoding='utf-8-sig')

print("\n🎉🎉🎉 성공! 'training_data_10k.csv' 파일이 생성되었습니다! 🎉🎉🎉")
print(f"총 {len(final_df)}개의 균형 잡힌 학습 데이터가 만들어졌습니다.")

⏳ 1단계: 원본 CSV 파일들을 불러옵니다...
✅ 파일 로드 완료!
  - 긍정 데이터 생성 중 (포함 관계 확인)...


긍정 데이터 검색 중: 100%|██████████| 997/997 [3:47:48<00:00, 13.71s/it]


  - ✅ 긍정 데이터 5539개 생성 완료!
  - 부정 데이터 생성 중 (목표: 5539개)


부정 데이터 생성 중: 100%|██████████| 5539/5539 [00:00<00:00, 28063.87it/s]


  - ✅ 부정 데이터 5539개 생성 완료!

⏳ 3단계: 'training_data_10k.csv' 파일로 저장합니다...

🎉🎉🎉 성공! 'training_data_10k.csv' 파일이 생성되었습니다! 🎉🎉🎉
총 11078개의 균형 잡힌 학습 데이터가 만들어졌습니다.
